# Web scrapping for Japanese Mountains

We scrape information about 100 Famous Japanese Mountains from website https://www.peakbagger.com/list.aspx?lid=5651 in Python,
then visualize them in Tableau.

In [2]:
import requests, time, random, re
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
url_domain = 'https://www.peakbagger.com/'
url_hp = url_domain + 'list.aspx?lid=5651'
html_hp = requests.get(url_hp).content
soup_hp = BeautifulSoup(html_hp, 'html.parser')

In [4]:
print(soup_hp.prettify())

<!DOCTYPE html>
<html xmlns="https://www.w3.org/1999/xhtml">
 <head>
  <title>
   100 Famous Japanese Mountains - Peakbagger.com
  </title>
  <link href="pb.css" rel="stylesheet" type="text/css"/>
  <link href="favicon.ico?" rel="shortcut icon" type="image/x-icon"/>
  <script type="text/javascript">
   function gfi()
				{document.Form1.ProcessBox.value = "GoForIt"}
  </script>
 </head>
 <body style="margin-left:0;margin-top:0">
  <form action="./list.aspx?lid=5651" id="Form1" method="post" name="Form1">
   <input id="__VIEWSTATE" name="__VIEWSTATE" type="hidden" value="/wEPDwULLTE0MDUxMjI1NTUPFgIeE1ZhbGlkYXRlUmVxdWVzdE1vZGUCARYCZg9kFgICBw8WAh4HVmlzaWJsZWgWAmYPZBYCZg9kFgICAQ9kFgJmD2QWAgIDDxBkZBYAZGTy+WGMUQIMWn1JJwE6M1YJ9US96wZfV9piACNO/UlACw=="/>
   <script src="/ScriptResource.axd?d=bE-ehdLCm5_hockRWkACFeqaOcSu2vVPFo-YwVPUN34KRPY900IzrYzsWb7BhUNcTU0xI4i6YyRlrK0-cvgEPbdLvI8EK88WNN0mEBKEEk6iDFVQOdswNrLme7QF3aERZ1sYUYLpKPzti5mfe4js2brQY4eylXaOo041VmT0mSA1&amp;t=2fe674eb" type="text/javas

In [ ]:
# Test for peak 1

In [5]:
peaks = soup_hp.select('h1 + table tr')[2:]
peak = peaks[0]

print(len(peaks))
peak

100


<tr><td>1.</td><td><a href="peak.aspx?pid=10882">Fuji-san</a></td><td>Kanto</td><td align="right">3776</td><td>Yamanashi-ken/Shizuoka-ken</td><td><a href="range.aspx?rid=563">Honshu</a></td><td align="right">3776</td><td style="text-align:center">318</td></tr>

In [7]:
name = peak.a.text
region = peak.select('td:nth-child(3)')[0].text
elev = float(peak.select('td:nth-child(4)')[0].text)
url_sub = peak.a['href']
url = url_domain + url_sub
print(name, region, url_sub)

Fuji-san Kanto peak.aspx?pid=10882


In [8]:
requests.packages.urllib3.disable_warnings()
html = requests.get(url, verify = False).text

In [9]:
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="https://www.w3.org/1999/xhtml">
 <head>
  <title>
   Fuji-san - Peakbagger.com
  </title>
  <meta content="Microsoft Visual Studio .NET 7.1" name="GENERATOR"/>
  <meta content="C#" name="CODE_LANGUAGE"/>
  <meta content="JavaScript" name="vs_defaultClientScript"/>
  <meta content="http://schemas.microsoft.com/intellisense/ie5" name="vs_targetSchema"/>
  <link href="pb.css" rel="stylesheet" type="text/css"/>
  <link href="favicon.ico?" rel="shortcut icon" type="image/x-icon"/>
  <script type="text/javascript">
   function gfi()
				{ document.Form1.ProcessBox.value = "GoForIt" }

				function Center(plati, plongi, lati, longi, peakid, layer, hj) {
				    var ifr = document.getElementById('Gmap')
   				    
				    var source = "map/MasterMap.aspx?sy=" + plati +
				    "&sx=" + plongi + "&z=15&t=P&d=" + peakid + "&a=1&cy=" +
				    lati + "&cx=" + lon

In [22]:
latlong = soup.select('tr:nth-child(4) > td:nth-child(2)')[0].text
print(latlong)
lat = float(re.findall('E([0-9.]+),', latlong)[0])
long = float(re.findall(', ([0-9.]+) \(Dec Deg\)', latlong)[0])

print(lat, long)

35° 21' 38'' N, 138° 43' 38'' E35.360638, 138.727347 (Dec Deg)293513 E 3915408 N, Zone 54 (UTM)
35.360638 138.727347


In [12]:
print(name, region, elev, lat, long, url_sub)

Fuji-san Kanto 3776.0 35.360638 138.727347 peak.aspx?pid=10882


In [13]:
df = pd.DataFrame(columns=['no.', 'name', 'region', 'elev', 'lat', 'long', 'url'])
df.loc[0] = [0+1, name, region, elev, lat, long, url_sub]
df

,no.,name,region,elev,lat,long,url
0,1,Fuji-san,Kanto,3776.0,35.360638,138.727347,peak.aspx?pid=10882


In [14]:
print('Row {} done ==> Mountain name: {}'.format(0+1, name))
time.sleep(random.random())
print('s')

Row 1 done ==> Mountain name: Fuji-san
s


In [ ]:
# Extract information for all 100 peaks

In [42]:
df = pd.DataFrame(columns=['no.', 'name', 'region', 'elev', 'lat', 'long', 'url'])
counter = 0

for peak in peaks:
    name = peak.a.text
    region = peak.select('td:nth-child(3)')[0].text
    elev = float(peak.select('td:nth-child(4)')[0].text)
    url_sub = peak.a['href']
    url = url_domain + url_sub
    
    requests.packages.urllib3.disable_warnings()
    html = requests.get(url, verify = False).text
    
    soup = BeautifulSoup(html, 'html.parser')
    
    latlong = soup.select('table')[3].text
    lat = float(re.findall('E([0-9.]+),', latlong)[0])
    long = float(re.findall(', ([0-9.]+) \(Dec Deg', latlong)[0])
    
    df.loc[counter] = [counter+1, name, region, elev, lat, long, url_sub]
    
    counter += 1
    time.sleep(random.random())
    if counter % 10 == 0:
        print('Row {} done ==> Mountain name: {}'.format(counter, name))
    
print(df)
df.to_csv('A3_mountains.csv', index=False)

Row 10 done ==> Mountain name: Senjoga-dake
Row 20 done ==> Mountain name: Washiba-dake
Row 30 done ==> Mountain name: Kinpu-san
Row 40 done ==> Mountain name: Hiuchiga-take
Row 50 done ==> Mountain name: Naeba-san
Row 60 done ==> Mountain name: Nishi-Azuma-yama
Row 70 done ==> Mountain name: Miyanoura-dake
Row 80 done ==> Mountain name: Kuju-san
Row 90 done ==> Mountain name: Hachimantai
Row 100 done ==> Mountain name: Tsukuba-san
    no.         name         region    elev        lat        long  \
0     1     Fuji-san          Kanto  3776.0  35.360638  138.727347   
1     2    Kita-dake          Chubu  3192.0  35.674537  138.238833   
2     3  Hotaka-dake          Chubu  3190.0  36.289203  137.647986   
3     4    Aino-dake          Chubu  3189.0  35.646037  138.228292   
4     5  Yariga-take          Chubu  3180.0  36.341980  137.647625   
..  ...          ...            ...     ...        ...         ...   
95   96  Meakan-dake       Hokkaido  1499.0  43.386433  144.008627   
96  